# Extraire la legende depuis la description
## tentative n°1

https://regex101.com/

In [711]:
import pickle
import regex   # permet overlapping matching

In [713]:
def print_legend( legend ):
    sortfun = lambda x: ( x['number'] , x['label'][::-1] )
    legendsorted = sorted( legend, key=sortfun  )

    current_number = legendsorted[0]['number']
    for match in legendsorted:
        if current_number != match['number']:
            print('     --')
            current_number = match['number']

        n_merged = len(match['position'])
        n_merged_str = '  ' if n_merged==1 else '(%i)'%n_merged
        print( '{label:>42} {number:>4}  {n}  {context:>30}'.format( **match, n=n_merged_str) )

In [714]:
# load pre-processed data
data = pickle.load( open( "../web/data/patent_infos.pickle", "rb" ) )
len(data)

379

In [715]:
# select a patent

k = 144 #41
patent = list(data.values())[k]
description = patent['description']

print( patent['patent_number'] )
print( 'https://www.google.com/patents/%s'%patent['patent_number'] )

print('nombre de mots :', len(description.split(' ')))

US20120210583
https://www.google.com/patents/US20120210583
nombre de mots : 10581


In [716]:
# Travail sur le texte entier :

description = description.replace('\n', '')
description = regex.sub('\s+',' ', description) # remove multiple space
description = description.lower()

# patent number XX
patent_number_pattern = r'(\d{1, 4}[,-])?\d{3, 8}[,-]\d{3, 8}'
description = regex.sub(patent_number_pattern, '', description  )

In [717]:
# Find values
units = ['inch', 'volt', 'm', 'mm']
find_sci_values =  r" (\d{1,4}([\.,]\d{1,4})? (%s)\W)" % '|'.join(units)

print( regex.findall(find_sci_values, description) )
description = regex.sub(find_sci_values, '<value>', description  )

[('0.3 mm ', '.3', 'mm'), ('0.3 mm ', '.3', 'mm'), ('0.3 mm ', '.3', 'mm')]


In [718]:
betweenpattern = r"(between [\d\.,]{1,4} to [\d\.,]{1,4}\W)"

print( regex.findall(betweenpattern, description) )
description = regex.sub(betweenpattern, '<between>', description  )

[]


In [719]:
description

'cross-reference to related applications [0001] this application is a continuation of pct international application number pct/kr2010/006528 filed on sep. 27, 2010, which claims priority of korean patent application number 10-2009-0096208 filed on oct. 9, 2009. the disclosure of each of the foregoing applications is incorporated herein by reference in its entirety. technical field [0002] the present invention relates to a nail clipper, and more particularly, to a method for manufacturing an inclined curve-type blade body, a blade body made therefrom, and a nail clipper having the blade body thereof which has a 3-dimensional structure complying with the curvature of a user&#39;s nail and is also capable of mass production of the same having the inclined curve-type blades exactly engaged with each other regardless of a grinding deviation. background art [0003] in general, a conventional nail clipper is provided with upper and lower blades constituting a blade body, which are in the form 

In [720]:
# trouve tout les chiffres, avec un espace devant
#exp =  r"(.{40}) (\d{1,3})"

exp =  r" (?P<n1>\d{1,3})( (and|or) (?P<n2>\d{1,3}))?\W"

allmatches = list( regex.finditer(exp, description) )
print(len(allmatches))

legend = []
for m in allmatches:

    label = description[ max(0, m.start()-40 ): m.start() ]  # to prevent overlapping
    context = description[ max(0, m.start()-10 ): min( len(description), m.end()+10 ) ]  # to debug
    
    item = {'number':int(m.group('n1')), 'label':label, 'position':[m.span('n2')], 'context':context}
    legend.append( item )
    
    if m.group('n2'): # correspond to "23 and 24"
        item = {'number':int(m.group('n2')), 'label':label, 'position':[m.span('n2')], 'context':context}
        legend.append( item )
        
print(len(legend))

480
548


In [721]:
print_legend( legend[:10] )

  r blades for various nail clippers. fig.    1             pers. fig. 1 shows cons
  seen from the plan view as shown in fig.    1             wn in fig. 1( d ), so a
   in the x axis. [0005] referring to fig.    1             ng to fig. 1 , the uppe
  ly engaged with each other. [0004] figs.    1             004] figs. 1 to 5 are s
     --
  point c is the center of the blade. fig.    2             lade. fig. 2 shows cons
     --
  aving the blade body thereof which has a    3             hich has a 3-dimensiona
     --
  gaged with each other. [0004] figs. 1 to    5             figs. 1 to 5 are schema
     --
  ion number 10-2009-0096208 filed on oct.    9             ed on oct. 9, 2009. the
     --
  rity of korean patent application number   10            ion number 10-2009-00962
     --
  n number pct/kr2010/006528 filed on sep.   27            ed on sep. 27, 2010, whi


In [722]:
def disqualify( legend, pattern, n=999 ):
    for i, row in enumerate( legend ):
        if pattern.search( row['label'] ):
            legend[ i ]['number'] = n

# Disqualifie si point final :  & tiret   ... mais pas une virgule car 45, 46 and 47 ...
# Fig. 1  ;  No. 511  
pattern = regex.compile( r'[\.-]$$' )

disqualify( legend, pattern, n=999 )

        
# si c'est un mois, ou un autre mots clés :
keywords = ['january', 'february', 'march', 'april', 'may', 'june',\
          'july', 'august', 'september', 'october', 'november', 'december',\
           'figure', 'claim', 'at', 'and']

keywords.extend( ['the', 'a', 'an', 'these', 'their', 'when', 'with', 'by', 'this', \
               'have', 'having', 'has', 'is', 'over', 'its', 'of said', 'and', 'as',\
              'of', 'in', 'to'] )

pattern = regex.compile( r'\W(%s)$' % '|'.join(keywords), regex.I )

disqualify( legend, pattern, n=998 )
        
    
print_legend( legend )

  of the fig. 15 are supposed to be ground    0              be ground 0.3 min deep
     --
  erformed in a short period of time about    1             time about 1-2 seconds 
   corresponding to the angle θ 1 (here, θ    1             1 (here, θ 1 =θ 2 ) to 
  d surface having a predetermined angle θ    1             ed angle θ 1 with respe
  e angle θ 2 corresponding to the angle θ    1             he angle θ 1 of the upp
  e angle θ 2 corresponding to the angle θ    1             he angle θ 1 of the upp
  e angle θ 2 corresponding to the angle θ    1             he angle θ 1 of the upp
  e angle θ 2 corresponding to the angle θ    1             he angle θ 1 of the upp
  e angle θ 2 corresponding to the angle θ    1             he angle θ 1 (here, θ 1
  in fig. 15( a ), the inclination angle θ    1             on angle θ 1 in the upp
  d. that is, when the inclination angle θ    1             on angle θ 1 of the upp
  he conditions on the inclination angle θ    1             on angle

  eparated from the lower curve-type blade   54            type blade 54 indicated 
  earward inclined convex curve-type blade   54            type blade 54 (refer to 
   wherein the opposite ends of the blades   54            the blades 54 make conta
  eanwhile, the inclined curve-type blades   54      ype blades 53 and 54 of the pre
  ing work, the inclined curve-type blades   54      ype blades 53 and 54 having the
   that is, the inclined curve-type blades   54      ype blades 53 and 54 which are 
  body 22 . the inclined curve-type blades   54      ype blades 53 and 54 in accorda
  s because the inclined curve-type blades   54      ype blades 53 or 54 having the
  30 having the inclined curve-type blades   54      ype blades 53 and 54 . hence, t
   applying the inclined curve-type blades   54      ype blades 53 and 54 in accorda
  tion with the inclined curve-type blades   54      ype blades 53 and 54 . at this 
  , wherein the inclined curve-type blades   54      ype blades 53 or

## Coupe sur certain mot clé (the, a, an, ...)

In [723]:
def cut_label( legend, pattern ):
    for i, row in enumerate( legend ):
        label = row['label']

        match = pattern.search(label)

        if match:
            newlabel = match.group(2)
            legend[i]['label'] = newlabel

In [724]:
small_words = ['the', 'a', 'an', 'these', 'their', 'when', 'with', 'by', 'this', \
               'have', 'having', 'has', 'is', 'over', 'its', 'of said', 'and', 'as',\
              'of', 'in', 'to']

pattern = regex.compile( r'^.* (%s) (.+)$' % '|'.join(small_words), regex.I )

cut_label( legend, pattern )

In [725]:
# cut at point and comma :
pattern = regex.compile( '^.*([\.,;] )(.+)$' )

cut_label( legend, pattern )

print_legend( legend )

                                 be ground    0              be ground 0.3 min deep
     --
                                time about    1             time about 1-2 seconds 
                                         θ    1             1 (here, θ 1 =θ 2 ) to 
                                   angle θ    1             he angle θ 1 of the upp
                                   angle θ    1             he angle θ 1 of the upp
                                   angle θ    1             he angle θ 1 of the upp
                                   angle θ    1             he angle θ 1 of the upp
                                   angle θ    1             he angle θ 1 (here, θ 1
                     predetermined angle θ    1             ed angle θ 1 with respe
                       inclination angle θ    1             on angle θ 1 of the upp
                       inclination angle θ    1             on angle θ 1 , the widt
                       inclination angle θ    1             on angle

In [726]:
def get_positions( label, candidats ):
    positions = []
    for item in candidats:
        if item['label']==label:
            positions.extend( item['position'] )
    return positions

def get_context( label, candidats ):
    items = [ item for item in candidats if item['label']==label ]
    
    return items[0]['context']

def merge( legend ):
    # merge identical label
    
    numbers_unique = { item['number'] for item in legend  } # set

    new_legend = []
    for n in numbers_unique:

        candidats = [ line for line in legend if line['number']==n ]

        labels_unique = { item['label'] for item in candidats  }

        new_candidats = [ { 'number':n, 'label':label_u,\
                           'position':get_positions(label_u, candidats), 'context':get_context(label_u, candidats) }
                          for label_u in labels_unique ]

        new_legend.extend( new_candidats )

    return new_legend

In [727]:
print_legend(  merge( legend ) )

                                 be ground    0              be ground 0.3 min deep
     --
                                time about    1             time about 1-2 seconds 
                                         θ    1             1 (here, θ 1 =θ 2 ) to 
                                   angle θ    1  (5)         he angle θ 1 of the upp
                     predetermined angle θ    1             ed angle θ 1 with respe
                       inclination angle θ    1  (5)         on angle θ 1 of the upp
                      inclination angles θ    1             n angles θ 1 and θ 2 (θ
                 same inclination angles θ    1             n angles θ 1 and θ 2 wi
                                    θ 2 (θ    1             and θ 2 (θ 1 =θ 2 ) are
     --
                                         θ    2  (2)          θ 1 and θ 2 with respe
                                   angle θ    2  (6)         an angle θ 2 correspond
                                    θ 1 =θ    2         

### To look
* sort of recursive... identifier un permet d'en déduire un autre...
* october... months

## other things
* Find values like "110 volts"... 